In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [ ]:
data = spark.read.csv('customer_churn.csv', inferSchema = True, header = True)

In [ ]:
data.printSchema()

In [ ]:
data.describe().show()

In [ ]:
data.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols = ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], outputCol = 'features')

In [ ]:
output = assembler.transform(data)

In [ ]:
final_data = output.select('features', 'churn')

In [ ]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr_churn = LogisticRegression(labelCol = 'churn')

In [ ]:
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary

In [ ]:
training_sum.predictions.describe().show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [ ]:
pred_and_labels.predictions.show()

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.prediction)
auc

In [ ]:
### predict on new data

In [ ]:
final_lr_model = lr_churn.fit(final_data)

In [ ]:
new_customers = spark.read.csv('new_customers.csv', inferSchema = True, header = True)

In [ ]:
new_customers.printSchema()

In [ ]:
test_new_customers = assembler.transform(new_customers)

In [ ]:
test_new_customers.printSchema()

In [ ]:
final_results = final_lr_model.transform(new_customers)
final_results

In [ ]:
final_results.select('Company', 'prediction').show()

In [ ]:
test_new_customers.describe().show()